# Anagram maker
This notebook is a testing ground for trying out different ways to make a good-sounding anagram using the stats computed in `compute_stats.ipynb`.

In [ ]:
import json
import random

from marvolo import atomize, join_groups

In [ ]:
with open("stats/names-female.json", "r") as file:
    stats = json.load(file)
stats.keys()

In [ ]:
def log_name_probability(name):
    grouped_name = atomize(name)
    for popular_group in stats["popular_groups"]:
        grouped_name = join_groups(grouped_name, popular_group)
    try:
        return (
            stats["log_group_probabilities"][grouped_name[0]]
            + sum(
                stats["log_pair_probabilities"][first_group + second_group] - stats["log_group_probabilities"][first_group]
                for first_group, second_group in zip(grouped_name[:-1], grouped_name[1:])
            )
            + stats["log_length_probabilities"][str(len(grouped_name))]
        )
    except KeyError as e:
        return -float("inf")

log_name_probability("KOWALSKI")

In [ ]:
def random_swap(text: str):
    indices = [random.randrange(len(text)) for _ in range(2)]
    low_idx = min(indices)
    high_idx = max(indices)
    if low_idx == high_idx:
        return text
    return text[:low_idx] + text[high_idx] + text[low_idx + 1:high_idx] + text[low_idx] + text[high_idx + 1:]

random_swap("KOWALSKI")

In [ ]:
def shuffle(text: str):
    result = list(text)
    random.shuffle(result)
    return "".join(result)

shuffle("KOWALSKI")

In [ ]:
def suggest_swap(text: str, num_tries=100):
    return max(
        (random_swap(text) for _ in range(num_tries)),
        key=log_name_probability,
    )

suggest_swap("KOWLASKI")

In [ ]:
def suggest_anagram(name: str, num_iterations=100):
    anagram = shuffle(name)
    for _ in range(num_iterations):
        anagram = suggest_swap(anagram)
    return anagram

suggest_anagram("KOWALSKI")